In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.listdir('/kaggle/input/petfinder-pawpularity-score')
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
submit = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
train = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')

In [ ]:
train.head()

In [ ]:
train.info(), test.info(), submit.info()

In [ ]:
import os

_train_path = '/kaggle/input/petfinder-pawpularity-score/train/'
_test_path = '/kaggle/input/petfinder-pawpularity-score/test/'

train_imgs = os.listdir(_train_path)
test_imgs = os.listdir(_test_path)

#from PIL import Image 
#img = Image.open(_train_path + train_imgs[0])
#img.show()

from IPython.display import Image
Image(filename=_train_path + train_imgs[0])

In [ ]:
len(train_imgs), len(test_imgs)

In [ ]:
# Import libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
# Import Warnings 
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
# Import tensorflow as the backend for Keras
from keras import backend as K
#K.set_image_dim_ordering('tf')
K.set_image_data_format('channels_first')
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
#from keras.optimizers import SGD, RMSprop, adam
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import TensorBoard
# Import required libraries for cnfusion matrix
from sklearn.metrics import classification_report,confusion_matrix
import itertools

In [ ]:
PATH = os.getcwd()
# Define data path
data_path = _train_path[:-6]
data_dir_list = os.listdir(data_path)
data_dir_list

In [ ]:
from tqdm import tqdm
import time

img_rows=128
img_cols=128
num_channel=3
num_epoch=100
# Define the number of classes
num_classes = 1
img_data_list=[]
'''for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize=cv2.resize(input_img,(128,128))
        img_data_list.append(input_img_resize)
'''
start = time.time()
for img in tqdm(train_imgs):
    input_img=cv2.imread(_train_path + img )
    input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    input_img_resize=cv2.resize(input_img,(img_rows,img_cols))
    img_data_list.append(input_img_resize)

print(time.time()-start)
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print (img_data.shape)

In [ ]:
#K.common.image_dim_ordering()
K.image_data_format()

In [ ]:
if num_channel==1:
    if K.image_data_format()=='channels_first':
        img_data= np.expand_dims(img_data, axis=1)
        print (img_data.shape)
    else:
        img_data= np.expand_dims(img_data, axis=4)
        print (img_data.shape)
else:
    if K.image_data_format()=='channels_first':
        img_data=np.rollaxis(img_data,1,1)
        print (img_data.shape)
    else:
        img_data=np.rollaxis(img_data,3,1)
        print (img_data.shape)

In [ ]:
display(img_data.shape)
img_data = img_data.reshape(9912, 1, 128, 128)
display(img_data.shape)

In [ ]:
num_classes = 1
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')
labels[0:9912]=0
#labels[9912:]=1
#labels[365:567]=1
#labels[567:987]=2
names = ['dogs']

In [ ]:
labels.shape

In [ ]:
#Y = np_utils.to_categorical(labels, num_classes)
Y = train['Pawpularity']
Y.head()

In [ ]:
x,y = shuffle(img_data,Y, random_state=2)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=55)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
print("X_train shape = {}".format(X_train.shape))
print("X_test shape = {}".format(X_test.shape))

In [ ]:
image = X_train[1203,:].reshape((128,128))
plt.imshow(image)
plt.show()

In [ ]:
#img_data.shape
#img_data = img_data.reshape(9912, 1, 128, 128)
#[[1] + list(img_data[0].shape)][0]
#(img_data.reshape([img_data.shape[0], img_data.shape[1]] + list(img_data.shape[1:]))).shape
img_data[0].shape

In [ ]:
#Initialising the input shape
#X.shape here as I guess is something similar to the mnist data, (60000, 28, 28), means it doesn't have extra dimension or say 24bit-representation, 
#i.e., some color-bytes. As such, each x in X is having 2D shape, thus, X.shape[1:] -eq x.shape -eq (28, 28). 
#You have to explicitly reshape X to include the extra dimension needed for Conv2D layer.
#
#As per the code, seems you want to use 'channel_last' configuration, for which reshaping of X_train and X_test may go like:
#
#X = X.reshape(list(X.shape) + [1])    # (60000, 28, 28, 1)
#For 'channel_first' it will be:
#
#X = X.reshape([X.shape[0], [1]] + list(X.shape[1:]))    # (60000, 1, 28, 28)

#Hope it helps the reader.
input_shape=img_data[0].shape
#input_shape=[[1] + list(img_data[0].shape)][0]
# Design the CNN Sequential model
cnn_model = Sequential([
    Convolution2D(32,3,3,padding ='same',activation='relu',input_shape = input_shape),
    Convolution2D(32,3,3,activation='relu'),
    MaxPooling2D(pool_size=(2,2)) ,
    Dropout(0.5),
    Flatten(), 
    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(num_classes)
])

In [ ]:
#cnn_model.compile(loss='categorical_crossentropy', optimizer='adadelta',metrics=["accuracy"])
cnn_model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8),metrics=["mse", "msle"])

In [ ]:
cnn_model.summary()

In [ ]:
X_test.shape, y_train.shape, y_train.head()

In [ ]:
start = time.time()
hist = cnn_model.fit(X_train, y_train, batch_size=16, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test))
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_mse=hist.history['mse']
val_mse=hist.history['val_mse']
train_msle=hist.history['msle']
val_msle=hist.history['val_msle']
xc=range(num_epoch)
print(time.time() - start)

In [ ]:
valid_img_list=[]

start = time.time()
for img in tqdm(test_imgs):
    input_img=cv2.imread(_test_path + img )
    input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    input_img_resize=cv2.resize(input_img,(img_rows,img_cols))
    valid_img_list.append(input_img_resize)

print(time.time()-start)
valid_img_data = np.array(valid_img_list)
valid_img_data = valid_img_data.astype('float32')
valid_img_data /= 255

valid_img_data = valid_img_data.reshape(8, 1, 128, 128)
print ('valid_img_data', valid_img_data.shape)

In [ ]:
if num_channel==1:
    if K.image_data_format()=='channels_first':
        valid_img_data= np.expand_dims(valid_img_data, axis=1)
        print (img_data.shape)
    else:
        valid_img_data= np.expand_dims(valid_img_data, axis=4)
        print (valid_img_data.shape)
else:
    if K.image_data_format()=='channels_first':
        valid_img_data=np.rollaxis(valid_img_data,1,1)
        print (valid_img_data.shape)
    else:
        valid_img_data=np.rollaxis(valid_img_data,3,1)
        print (valid_img_data.shape)

In [ ]:
y_paw_pred = cnn_model.predict(valid_img_data, verbose=0)

In [ ]:
y_paw_pred

In [ ]:
from sklearn.metrics import mean_squared_log_error

# mean_squared_log_error(y_true, y_pred, *, sample_weight=None, multioutput='uniform_average', squared=True)

def rmlse(y_true, y_pred):
  rmsle_ = np.sqrt(mean_squared_log_error(y_true, y_pred))
  print(f'RMLSE :{rmsle_:.4f}')
  return rmsle_

In [ ]:
submit['Pawpularity'] = y_paw_pred
submit.to_csv('submission.csv')